## Install dependencies

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%pip install -e ../aicacia/extraction aicacia-document-exporter==0.1.4

## Extract from TEI files and populate DB

In [ ]:
from aicacia_document_exporter.Document import (Document, DocumentSection,
                                                MediaType)
from aicacia_document_exporter.PreprocessingModel import PreprocessingModel
from aicacia_document_exporter.SimpleFileDocumentExporter import \
    SimpleFileDocumentExporter
from aicacia_extraction.grobid import TEIDocument

In [ ]:
tei_document = TEIDocument("../data/grobid/sample.grobid.tei.xml")

In [ ]:
tei_document.title